In [1]:
import argparse
import os

import lib.medloaders as medical_loaders
import lib.medzoo as medzoo
# Lib files
import lib.utils as utils
from lib.utils.early_stopping import EarlyStopping
from lib.utils.general import prepare_input
from lib.losses3D import DiceLoss, create_loss,BCEDiceLoss

import torch
import numpy as np

from easydict import EasyDict

from lib.metric3D.DiceCoefficient import DiceCoefficient
from lib.metric3D.MeanIoU import MeanIoU

from tqdm import tqdm

import time

import shutil

import matplotlib.pyplot as plt

In [2]:
args = EasyDict({
    "batch_size" : 1,
    "test_name" : "MICCAI-2008-MSLESION-3DUnet-Flair",
    "dataset_path" : r"datasets\MICCAI_2008_MS_Lesions", 
    "dim" : (144,144,144),
    "nEpochs" : 100,
    "classes" : 1,
    "split" : (0.8, 0.1 , 0.1),
    "inChannels" : 1,
    "inModalities" : 1,
    "fold_id" : '1',
    "lr" : 1e-4,
    "cuda" : True,
    "resume" : '',
    "model" : 'UNET3D', # VNET VECT2 UNET3D DENSENET1 DENSENET2 DENSENET3 HYPERDENSENET
    "opt" : 'adam', # sgd adam rmsprop
    "log_dir" : 'runs',
    "loadData" : False,
    "terminal_show_freq" : 10,
})

start_time = time.strftime("%Y%m%d-%H%M%S", time.localtime(time.time()))

args.result_path = rf'results/{args.test_name}-{start_time}'

shutil.rmtree(args.result_path, ignore_errors=True)
utils.make_dirs(args.result_path)

args.save = rf'saved_models/{args.model}_checkpoints/{args.model}-{args.test_name}-{start_time}'
args.save_checkpoint = os.path.join(args.save,'checkpoint.pt')
args.tb_log_dir = rf'runs/{args.model}-{args.test_name}-{start_time}'

utils.make_dirs(args.tb_log_dir)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
## FOR REPRODUCIBILITY OF RESULTS
seed = 1777777
utils.reproducibility(args, seed)
utils.make_dirs(args.save)
utils.save_arguments(args, args.save)

# Tensor Board

In [4]:
from torch.utils.tensorboard import SummaryWriter

# 기본 `log_dir` 은 "runs"이며, 여기서는 더 구체적으로 지정하였습니다
writer = SummaryWriter(f'{args.tb_log_dir}/log')

# Augmentation

In [5]:
import torchio as tio

from torchio.transforms import (
    RandomFlip,
    RandomAffine,
    RandomElasticDeformation, 
    RandomNoise,
    RandomMotion,
    RandomBiasField,
    RescaleIntensity,
    Resample,
    ToCanonical,
    ZNormalization,
    CropOrPad,
    HistogramStandardization,
    OneOf,
    Compose,
)

from lib.augment3D.random_crop_or_pad import RandomCropOrPad

### Full-aug ###
transform = tio.Compose([
    RandomCropOrPad(args.dim),
    RandomMotion(p=0.2),
    RandomBiasField(p=0.3),
    RandomNoise(p=0.5),
    RandomFlip(axes=(0,)),
    RandomAffine(p=0.5),
    RandomElasticDeformation(p=0.5),
    ZNormalization(),
])

### selectiv-aug ###
# transform = tio.Compose([
#     CropOrPad((144,144,144)),
#     # RandomMotion(p=0.2),
#     # RandomBiasField(p=0.3),
#     RandomNoise(p=0.5),
#     RandomFlip(axes=(0,)),
#     # RandomAffine(p=0.5),
#     RandomElasticDeformation(p=0.5),
#     ZNormalization(),
# ])

validation_transform = tio.Compose([
    CropOrPad(args.dim),
    ZNormalization()
])

test_transform = tio.Compose([
    CropOrPad(args.dim),
    ZNormalization()
])


# Dataset

In [6]:
from lib.medloaders.miccai_2008_ms_lesions import MICCAI2008MSLESIONS
from lib.medloaders.cornel_ms_lesions import CORNEL_MS_LESIONS
from torch.utils.data import DataLoader

train_dataset = MICCAI2008MSLESIONS(train_mode='train', 
                                dataset_path=args.dataset_path,
                                classes=args.classes,
                                crop_dim=args.dim, 
                                split=args.split,
                                transform=transform,
                                sample_per_image=10)

val_dataset = MICCAI2008MSLESIONS(train_mode='val',
                                dataset_path=args.dataset_path,
                                classes=args.classes, 
                                crop_dim=args.dim, 
                                split=args.split,
                                transform=validation_transform,
                                sample_per_image=1)


test_dataset = MICCAI2008MSLESIONS(train_mode='test',
                                dataset_path=args.dataset_path,
                                classes=args.classes, 
                                crop_dim=args.dim, 
                                split=args.split,
                                transform=test_transform,
                                sample_per_image=1)

params = {
                'batch_size': args.batch_size,
                'shuffle': False,
                'num_workers': 4
        }

print(len(train_dataset))

train_generator = DataLoader(train_dataset, **params)
val_generator = DataLoader(val_dataset, **params)
test_generator = DataLoader(test_dataset, **params)

train : flair, mprage, t2w, pdw and label .nii.gz to .pt:   0%|          | 0/16 [00:00<?, ?it/s]

sampling:   0%|          | 0/10 [00:00<?, ?it/s]

# Model & Optimizer

In [ ]:
from lib.medzoo.ResUnet3D import ResUNet3D
from lib.medzoo.Unet3D import Unet3D
import torch.optim as optim

# model = ResUNet3D(in_channels=args.inChannels, n_classes=args.classes)
model = Unet3D(in_channels=args.inChannels, out_channels=args.classes)
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-10)

if torch.cuda.is_available():
    model = model.cuda()

# Loss Function

In [ ]:
criterion = BCEDiceLoss(alpha=1, beta=1)

In [ ]:
def train_model(model, train_generator, val_generator ,patience, n_epochs, metrics, tensorboard=None):

    # 모델이 학습되는 동안 trainning loss를 track
    train_losses = []
    # 모델이 학습되는 동안 validation loss를 track
    valid_losses = []
    # epoch당 average training loss를 track
    avg_train_losses = []
    # epoch당 average validation loss를 track
    avg_valid_losses = []

    train_metrics = [[] for i in range(len(metrics))]
    val_metrics = [[] for i in range(len(metrics))]

    # early_stopping object의 초기화
    early_stopping = EarlyStopping(patience = patience, verbose = True,path=args.save_checkpoint)

    n_epochs_length = len(str(n_epochs))
    for epoch in range(1, n_epochs + 1):
        ###################
        # train the model #
        ###################
        model.train() # prep model for training

        progressbar = tqdm(train_generator)
        for batch, input_tuple in enumerate(progressbar, start=1):
        
            # clear the gradients of all optimized variables
            optimizer.zero_grad()

            # gpu 연산으로 변경
            input, target = prepare_input(input_tuple=input_tuple, args=args)
            input.requires_grad = True

            # forward pass: 입력된 값을 모델로 전달하여 예측 출력 계산
            output = model(input)

            output = output.cpu()
            target = target.cpu()

            # calculate the loss
            loss = criterion(output, target)

            # backward pass: 모델의 파라미터와 관련된 loss의 그래디언트 계산
            loss.backward()

            # perform a single optimization step (parameter update)
            optimizer.step()

            # record training loss
            train_losses.append(loss.item())

            output[output > 0.5] = 1.0

            # record matric
            for i,metric in enumerate(metrics):
                value = metric(output,target).item()
                train_metrics[i].append(value)

            # print metric & loss
            print_msg = {
                'loss' : f'{np.average(train_losses):.5f}',
            }

            for i,metric in enumerate(metrics):
                print_msg[metric.metric_name] = f'{np.average(train_metrics[i]):.5f}'

            progressbar.set_postfix(print_msg)
            progressbar.set_description(f'[{epoch:>{n_epochs_length}}/{n_epochs:>{n_epochs_length}}][{batch}/{len(train_generator)}]')

        ######################    
        # validate the model #
        ######################
        model.eval() # prep model for evaluation

        prediction_images = []

        with torch.no_grad():
            for i,input_tuple in enumerate(val_generator) :

                input, target = prepare_input(input_tuple=input_tuple, args=args)
                # input.requires_grad = True

                # forward pass: 입력된 값을 모델로 전달하여 예측 출력 계산
                output = model(input)

                input = input.cpu()
                output = output.cpu()
                target = target.cpu()

                # calculate the loss
                loss = criterion(output, target)

                # record validation loss
                valid_losses.append(loss.item())

                # save prediction result
                prediction_images.append(
                    [input.detach().numpy(),output.detach().numpy(), target.detach().numpy()])

                output[output > 0.5] = 1.0

                # metric
                for i,metric in enumerate(metrics):
                    val_metrics[i].append(metric(output,target).item())

        columns = 3
        rows = len(generator) * args.batch_size

        fig=plt.figure(figsize=( 3 * columns,3 * rows))

        i = 1
        for [input,output, target] in prediction_images:

            for batch_idx in args.batch_size:
                fig.add_subplot(rows, columns, i); i+=1
                plt.imshow(input[0,0,:,:,input.shape[-1] // 2], cmap='gray')
                plt.title("original Image"); plt.axis('off')

                fig.add_subplot(rows, columns, i); i+=1
                plt.imshow(output[0,0,:,:, input.shape[-1] // 2], cmap='gray')
                plt.title("Predicited Image"); plt.axis('off')

                fig.add_subplot(rows, columns, i); i+=1
                plt.imshow(target[0,0,:,:, input.shape[-1] // 2], cmap='gray')
                plt.title("Original Mask"); plt.axis('off')

        plt.savefig(f'{args.result_path}/{epoch}.png')
        plt.clf()

        # print 학습/검증 statistics
        # epoch당 평균 loss 계산
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)

        # epoch당 평균 metric 계산
        train_metric = {}
        val_metric = {}

        for i, metric in enumerate(metrics):
            train_metric[metric.metric_name] = np.average(train_metrics[i])
            val_metric[metric.metric_name] = np.average(val_metrics[i])


        # epoch 결과 출력
        print_msg = (f'[{epoch:>{n_epochs_length}}/{n_epochs:>{n_epochs_length}}] ' +
                     f'loss: {train_loss:.5f} ' +
                     f'val_loss: {valid_loss:.5f} ')

        for key in train_metric.keys():
            print_msg += f'{key} : {train_metric[key]:.5f} '

        for key in val_metric.keys():
            print_msg += f'val_{key} : {val_metric[key]:.5f} '

        # tensorboard
        if tensorboard:
            writer.add_scalars("loss", 
                                {
                                    'train' : train_loss,
                                    'val' : valid_loss
                                 },
                                epoch)

            for train_key, val_key in zip(train_metric.keys(), val_metric.keys()):
                writer.add_scalars(train_key,
                                {
                                    'train' : train_metric[train_key],
                                     'val' : val_metric[val_key]
                                }, 
                                 epoch)

        # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        train_metrics = [[] for i in range(len(metrics))]
        val_metrics = [[] for i in range(len(metrics))]

        # early_stopping는 validation loss가 감소하였는지 확인이 필요하며,
        # 만약 감소하였을경우 현제 모델을 checkpoint로 만든다.
        early_stopping(valid_loss, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

   # best model이 저장되어있는 last checkpoint를 로드한다.
    model.load_state_dict(torch.load(args.save_checkpoint))
    writer.close()

    return  model, avg_train_losses, avg_valid_losses

In [ ]:
def evaluate(model, generator, tensorboard, metrics, args):
    model.eval() # prep model for evaluation

    prediction_images = []

    eval_losses = []
    eval_metrics = [[] for i in range(len(metrics))]

    with torch.no_grad():

        progressbar = tqdm(generator)

        for batch,input_tuple in enumerate(progressbar, start=1) :

            input, target = prepare_input(input_tuple=input_tuple, args=args)
            # input.requires_grad = True

            # forward pass: 입력된 값을 모델로 전달하여 예측 출력 계산
            output = model(input)

            input = input.cpu()
            output = output.cpu()
            target = target.cpu()

            # calculate the loss
            loss = criterion(output, target)

            # record validation loss
            eval_losses.append(loss.item())

            # save prediction result
            prediction_images.append(
                [input.detach().numpy(),output.detach().numpy(), target.detach().numpy()])

            output[output > 0.5] = 1.0

            # metric
            for i,metric in enumerate(metrics):
                eval_metrics[i].append(metric(output,target).item())

    columns = 3
    rows = len(generator) * args.batch_size

    fig=plt.figure(figsize=( 3 * columns,3 * rows))

    i = 1
    for [input,output, target] in prediction_images:

        for batch_idx in args.batch_size:
            fig.add_subplot(rows, columns, i); i+=1
            plt.imshow(input[batch_idx,0,:,:,input.shape[-1] // 2], cmap='gray')
            plt.title("original Image"); plt.axis('off')

            fig.add_subplot(rows, columns, i); i+=1
            plt.imshow(output[batch_idx,0,:,:, input.shape[-1] // 2], cmap='gray')
            plt.title("Predicited Image"); plt.axis('off')

            fig.add_subplot(rows, columns, i); i+=1
            plt.imshow(target[batch_idx,0,:,:, input.shape[-1] // 2], cmap='gray')
            plt.title("Original Mask"); plt.axis('off')

    plt.savefig(f'{args.result_path}/test.png')
    plt.clf()

    # print 학습/검증 statistics
    # epoch당 평균 loss 계산
    eval_loss = np.average(eval_losses)

    # epoch당 평균 metric 계산
    eval_metric = {}

    for i, metric in enumerate(metrics):
        eval_metric[metric.metric_name] = np.average(eval_metrics[i])


    # epoch 결과 출력
    print_msg = (f'loss: {eval_loss:.5f} ')

    for key in eval_metric.keys():
        print_msg += f'{key} : {eval_metric[key]:.5f} '

    # tensorboard
    if tensorboard:
        writer.add_scalars(
                            "loss", 
                            {
                                'eval' : eval_loss,
                            },
                            0)

        for eval_key in eval_metric.keys():
            writer.add_scalars(
                                eval_key,
                                {
                                    'eval' : eval_metric[eval_key],
                                }, 
                                0)


# Metric

In [ ]:
meanIoU = MeanIoU(metric_name="MeanIoU")
diceCoefficient = DiceCoefficient(metric_name="DiCE")

In [ ]:
model, train_loss, valid_loss = train_model(model = model,
                                            train_generator = train_generator,
                                            val_generator = val_generator,
                                            patience = 10,
                                            n_epochs = args.nEpochs,
                                            metrics=[diceCoefficient, meanIoU],
                                            tensorboard=writer)

In [ ]:
evaluate(model, generator=test_generator, tensorboard=writer, metrics=[diceCoefficient, meanIoU], args=args)